## Portfolio Project # 3
### Economic Sentiment Analysis with Twitter and FRED APIs

April 12, 2023


The goal for this project is to conduct sentiment analyses of the US economy using Twitter data. Twitter granted my developer account elevated access to the Twitter API. Scraped Tweets will be fed into the  Sentiment Analysis library TextBlob. Currently, elevated Twitter access only allows searching for the past 7 days of Tweets.

Initial questions to explore: What are the current overall sentiments of the economy? Are they positive or negative or neutral? How has it varied over time? Is there a correlation between economic sentiment and perhaps certain economic metrics such as stock market performance?  Economic data will be downloaded from the U.S. Federal Reserve's FRED API.  In the future once enough data is collected, the data will be fed into scikit-learn machine learning models to best fit a model and determine if any of the economic indicators have correlations with sentiment polarity as the target variable.

Skills: Python; Twitter API; FRED API; Tweepy; Pandas; TextBlob; Matplotlib; Seaborn; Plotly Dash; Data Munging; Feature Engineering.


References for documentation and starter code:
<br>1. Twitter API docs
<br>&emsp;&emsp; https://developer.twitter.com/en/docs/twitter-api/
<br>2. Federal Reserve Bank of St. Louis FRED API docs
<br>&emsp;&emsp; https://fred.stlouisfed.org/docs/api/fred/
<br>3. Getting started with the Twitter API v2 for academic research
<br>&emsp;&emsp; https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research
<br>4. A comprehensive guide for using the Twitter API v2 with Tweepy in Python
<br>&emsp;&emsp; https://dev.to/twitterdev/a-comprehensive-guide-for-using-the-twitter-api-v2-using-tweepy-in-python-15d9
<br>5. How to use the Twitter API v2 in Python using Tweepy
<br>&emsp;&emsp; https://www.youtube.com/watch?v=0EekpQBEP_8&t=758s
<br>6. Building queries for Search Tweets
<br>&emsp;&emsp; https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
<br>7. Twitter API v2 data dictionary
<br>&emsp;&emsp; https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet
<br>8. Introduction to Twitter data processing and storage on AWS
<br>&emsp;&emsp; https://dev.to/twitterdev/introduction-to-twitter-data-processing-and-storage-on-aws-1og
<br>9. A Complete Step by Step Tutorial on Sentiment Analysis in Keras and Tensorflow
<br>&emsp;&emsp; https://towardsdatascience.com/a-complete-step-by-step-tutorial-on-sentiment-analysis-in-keras-and-tensorflow-ea420cc8913f
<br>10. Keeping your dataset compliant
<br>&emsp;&emsp; https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/6b-labs-code-standard-python.md

### Dependencies

In [ ]:
# Dependencies to be deployed on AWS Lambda as an imported package

# Install Dash
%pip install dash
%pip install jupyter-dash

# Install Dash Bootstrap Components
%pip install dash-bootstrap-components

# Install texblob and necessary NLTK corpora
%pip install -U textblob 
# %python -m textblob.download_corpora # This does not work, is it needed?

# Install emoji
%pip install emoji --upgrade

# https://github.com/letsgoexploring/fredpy
%pip install fredpy

# Python library for AWS Lambda to allow connection to AWS RDS PostgreSQL database
%pip install psycopg2-binary


In [2]:
import tweepy
import config    # Passwords & keys for RDS, Twitter & FRED APIs.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import re
import emoji
import fredpy as fp
# import psycopg2 as ps # needed when deployed to AWS Lambda

### Download tweets, determine counts, assess polarity by day and add to a dataframe

In [5]:
# Function to clean tweets from emojis, links and line breaks and more
# https://github.com/Coding-with-Adam/Dash-by-Plotly/blob/master/Dash_More_Advanced_Shit/NLP/nlpDash.py
# He found it somewhere on Medium or online, I was unable to locate the source

def clean_tweets(txt):
    txt = emoji.replace_emoji(txt, replace='')
    txt = txt.replace(r'[^\x00-\x7F]+', '')
    txt = re.sub(r"RT[\s]+", "", txt)
    txt = txt.replace("\n", " ")
    txt = re.sub(" +", " ", txt)
    txt = re.sub(r"https?:\/\/\S+", "", txt)
    txt = re.sub(r"(@[A-Za-z0–9_]+)|[^\w\s]|#", "", txt)
    txt.strip()
    return txt

In [6]:
# search tweets with certain terms and analyze them

client = tweepy.Client(bearer_token=config.BEARER_TOKEN)
query = '"US Economy" OR "U.S. economy" OR "American economy" -is:retweet lang:en'

# search_recent_tweets has a max results limit of 100, using paginator to increase max
# https://gist.github.com/mGalarnyk/5b251ea41c3626d6965ac7d5006da432
tweets = tweepy.Paginator(client.search_recent_tweets,
                          query=query,
                          tweet_fields=['created_at'], 
                          max_results=100).flatten(limit=100000) # increase to 100K for production
# tweets above is a generator object and can only be iterated through once
# need to sort payload by dates

all_tweets = []

cleaned_tweets, polarity_scores = [], []
for tweet in tweets:
    # Clean the tweet
    cleaned = clean_tweets(tweet.text)
    p = TextBlob(cleaned).sentiment.polarity
    polarity_scores.append(p)
    cleaned_tweets.append(cleaned) #used for troubleshooting payload transformation    
    
    # https://towardsdatascience.com/collect-data-from-twitter-a-step-by-step-implementation-using-tweepy-7526fff2cb31
    parsed_tweet = {}
    parsed_tweet['date'] = tweet.created_at
    parsed_tweet['text'] = cleaned
    parsed_tweet['Polarity'] = p
    all_tweets.append(parsed_tweet)

df = pd.DataFrame(all_tweets)
df['date'] = pd.to_datetime(df['date']).dt.date  # Strip time and leave date
# Revome duplicates if there are any
# df = df.drop_duplicates("date" , keep='first')

print(len(polarity_scores))
avg_polrty = sum(polarity_scores) / len(polarity_scores)
print('Average polarity for past 7 days: ', round(avg_polrty, 3))

23667
Average polarity for past 7 days:  0.044


In [7]:
# This does not subtract from monthly tweet limit.
counts = client.get_recent_tweets_count(query=query, granularity='day')

from datetime import datetime

dates = []
tweets = []
for count in counts.data:
    date = datetime.strptime(count['end'][:10], '%Y-%m-%d')
    # Tweet payload includes 2 end dates for the current date with the 1st being 
    # the full day so the last date needs to be ignored.
    if date in dates:
        break
    else:
        dates.append(date)
        tweets.append(count['tweet_count'])
tups = tuple(zip(dates, tweets))
tups

((datetime.datetime(2023, 4, 7, 0, 0), 3048),
 (datetime.datetime(2023, 4, 8, 0, 0), 4541),
 (datetime.datetime(2023, 4, 9, 0, 0), 2593),
 (datetime.datetime(2023, 4, 10, 0, 0), 1642),
 (datetime.datetime(2023, 4, 11, 0, 0), 3562),
 (datetime.datetime(2023, 4, 12, 0, 0), 4119),
 (datetime.datetime(2023, 4, 13, 0, 0), 4156))

In [8]:
date_list = list(tups)
df_mentions = pd.DataFrame(date_list, columns=['date', 'Mentions'])
df_mentions

,date,Mentions
0,2023-04-07,3048
1,2023-04-08,4541
2,2023-04-09,2593
3,2023-04-10,1642
4,2023-04-11,3562
5,2023-04-12,4119
6,2023-04-13,4156


In [9]:
df_p = df.groupby('date', as_index=False).mean()
df_p['date'] = pd.to_datetime(df_p['date'])  # convert to same dtype as df_mentions['date']
df_p

,date,Polarity
0,2023-04-06,0.051416
1,2023-04-07,0.068329
2,2023-04-08,-0.053386
3,2023-04-09,0.033976
4,2023-04-10,-0.015850
5,2023-04-11,0.093745
6,2023-04-12,0.080882
7,2023-04-13,0.083077


In [10]:
df_dmp = df_mentions.merge(df_p, how='right', on='date')
df_dmp = df_dmp.set_index(df.columns[0])
df_dmp

,Mentions,Polarity
date,,
2023-04-06,NaN,0.051416
2023-04-07,3048.0,0.068329
2023-04-08,4541.0,-0.053386
2023-04-09,2593.0,0.033976
2023-04-10,1642.0,-0.015850
2023-04-11,3562.0,0.093745
2023-04-12,4119.0,0.080882
2023-04-13,4156.0,0.083077


In [11]:
# Slicing out first two and the last dates since they may not capture full days of tweets
df_dmp = df_dmp[2:-1]
df_dmp

,Mentions,Polarity
date,,
2023-04-08,4541.0,-0.053386
2023-04-09,2593.0,0.033976
2023-04-10,1642.0,-0.015850
2023-04-11,3562.0,0.093745
2023-04-12,4119.0,0.080882


### Download important economic metrics from FRED - Federal Reserve Economic Data - https://fred.stlouisfed.org/
#### "This product uses the FRED® API but is not endorsed or certified by the Federal Reserve Bank of St. Louis."

In [12]:
fp.api_key = config.FRED_KEY    # FRED key availabe at https://fred.stlouisfed.org/docs/api/api_key.html

In [13]:
# Selecting dates to download monthly economic data ...
#  limited to 8 days since current Twitter API access level allows 7 days search 
# Setting up time delay to reduce API call rate

from datetime import datetime, timedelta
from time import sleep

secs = 31
t = datetime.today()
d = t - timedelta(days=8)
days = [d.strftime('%Y-%m-%d'), t.strftime('%Y-%m-%d')]
days

['2023-04-04', '2023-04-12']

In [14]:
# Download economic metrics with time delay and custom API request to reduce API call rate
# See "Custom API querries" here: https://www.briancjenkins.com/fredpy/docs/build/html/fredpy_examples.html

# Function for custom API request with economic series ID and dates as string inputs > 
# ...returns df with dates and economic series values
def series_df(series_id, period):
    
    # Specify the API path
    path = 'fred/series/observations'

    # API request and specify desired parameter values for the API query
    data = fp.fred_api_request(api_key=fp.api_key,
                                path=path,
                                parameters={'series_id':series_id,
                                            'observation_start':period[0],
                                            'observation_end':period[1],
                                            'file_type':'json'})

    # Return results in JSON format
    data = data.json()

    # # Load data, deal with missing values, format dates in index, and set dtype
    df = pd.DataFrame(data['observations'], columns =['date','value'])
    df = df.replace('.', np.nan)
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')['value'].astype(float)
    return df

In [15]:
# Download daily economic data

# Dow Jones Industrial Average
djia = series_df('DJIA', days)

# NASDAQ Composite Index
sleep(secs)
nasdaq = series_df('NASDAQCOM', days)

# S&P 500
sleep(secs)
sp500 = series_df('SP500', days)

# Federal Funds Effective Rate (benchmark for U.S. interest rates)
sleep(secs)
dff = series_df('DFF', days)

# 5-Year Breakeven Inflation Rate
sleep(secs)
infl = series_df('T5YIE', days)

# US benchmark for crude oil prices using West Texas Intermediate
sleep(secs)
tex = series_df('DCOILWTICO', days)

# 10 Year T-Note
sleep(secs)
Tnote = series_df('DGS10', days)

In [16]:
# Selecting dates to download monthly economic data

f = datetime.today()
s = f - timedelta(days=75)
month = [s.strftime('%Y-%m-%d'), f.strftime('%Y-%m-%d')]
month

['2023-01-27', '2023-04-12']

In [17]:
# Download monthly economic data

# Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
sleep(secs)
CPI_i = series_df('CPIAUCSL', month)

# Consumer Price Index for All Urban Consumers: 
    # Purchasing Power of the Consumer Dollar in U.S. City Average
sleep(secs)
CPI_p = series_df('CUUR0000SA0R', month)

In [18]:
# Functions for adding missing dates to dfs, usually weekends/holidays, and returns updated dfs

def missed_daily(series_id, col_name):
    
    # Set range of dates in accordance to prior API calls
    idx = pd.date_range(d.strftime('%Y-%m-%d'), t.strftime('%Y-%m-%d'))
    
    # Fill in missing dates and NaNs for their values
    series_id = pd.Series(series_id, name=col_name)
    series_id.index = pd.DatetimeIndex(series_id.index)
    df = series_id.reindex(idx)
    return df

def missed_monthly(series_id, col_name):
    
    # Set range of dates in accordance to prior API calls
    idx = pd.date_range(d.strftime('%Y-%m-%d'), t.strftime('%Y-%m-%d'))
    
    # Fill in missing dates and NaNs for their values
    series_id = pd.Series(series_id, name=col_name)
    series_id.index = pd.DatetimeIndex(series_id.index)
    df = series_id.reindex(idx, method='ffill') # forward fill into the days window for the monthly series
    return df

In [19]:
# Add missing dates to current economic series dfs

# Dow Jones Industrial Average
dj = missed_daily(djia, 'Dow')

# NASDAQ Composite Index
n = missed_daily(nasdaq, 'NASDAQ')

# S&P 500
sp = missed_daily(sp500, 'SP500')

# Federal Funds Effective Rate (benchmark for U.S. interest rates)
ff = missed_daily(dff, 'Interest')

# 5-Year Breakeven Inflation Rate
inf = missed_daily(infl, 'Inflation_5yr')

# US benchmark for crude oil prices using West Texas Intermediate
oil = missed_daily(tex, 'Oil')

# 10 Year T-Note
tnote = missed_daily(Tnote, 'T_Note_10yr')

# Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
cpi_i = missed_monthly(CPI_i, 'CPI_item_cost')

# Consumer Price Index for All Urban Consumers: Purchasing Power 
cpi_p = missed_monthly(CPI_p, 'CPI_purchase_power')

In [20]:
# Using Pandas concat else dates with NaN values are dropped when creating df
# Data that changes daily are on the left while monthly are on the right of df

econ = pd.concat((dj, n, sp, inf, oil, ff, tnote, cpi_i, cpi_p), axis=1)

df_econ = pd.DataFrame(index=dj.index.union(n.index), data=econ)
df_econ

,Dow,NASDAQ,SP500,Inflation_5yr,Oil,Interest,T_Note_10yr,CPI_item_cost,CPI_purchase_power
2023-04-04,33402.38,12126.33,4100.60,2.32,80.70,4.83,3.35,301.808,33.1
2023-04-05,33482.72,11996.86,4090.38,2.29,80.69,4.83,3.30,301.808,33.1
2023-04-06,33485.29,12087.96,4105.02,2.31,80.70,4.83,3.30,301.808,33.1
2023-04-07,NaN,NaN,NaN,2.33,NaN,4.83,3.39,301.808,33.1
2023-04-08,NaN,NaN,NaN,NaN,NaN,4.83,NaN,301.808,33.1
2023-04-09,NaN,NaN,NaN,NaN,NaN,4.83,NaN,301.808,33.1
2023-04-10,33586.52,12084.35,4109.11,2.34,79.79,4.83,3.41,301.808,33.1
2023-04-11,33684.79,12031.88,4108.94,2.36,NaN,4.83,3.43,301.808,33.1
2023-04-12,33646.50,NaN,4091.95,2.29,NaN,NaN,NaN,301.808,33.1


In [21]:
# Forward fill prior to analysis or visualizing

df_econ = df_econ.fillna(method='ffill')
df_econ

,Dow,NASDAQ,SP500,Inflation_5yr,Oil,Interest,T_Note_10yr,CPI_item_cost,CPI_purchase_power
2023-04-04,33402.38,12126.33,4100.60,2.32,80.70,4.83,3.35,301.808,33.1
2023-04-05,33482.72,11996.86,4090.38,2.29,80.69,4.83,3.30,301.808,33.1
2023-04-06,33485.29,12087.96,4105.02,2.31,80.70,4.83,3.30,301.808,33.1
2023-04-07,33485.29,12087.96,4105.02,2.33,80.70,4.83,3.39,301.808,33.1
2023-04-08,33485.29,12087.96,4105.02,2.33,80.70,4.83,3.39,301.808,33.1
2023-04-09,33485.29,12087.96,4105.02,2.33,80.70,4.83,3.39,301.808,33.1
2023-04-10,33586.52,12084.35,4109.11,2.34,79.79,4.83,3.41,301.808,33.1
2023-04-11,33684.79,12031.88,4108.94,2.36,79.79,4.83,3.43,301.808,33.1
2023-04-12,33646.50,12031.88,4091.95,2.29,79.79,4.83,3.43,301.808,33.1


In [22]:
# Merge tweets dataframe with economic metrics dataframe

df_final = df_dmp.join(df_econ, how='inner')
df_final

,Mentions,Polarity,Dow,NASDAQ,SP500,Inflation_5yr,Oil,Interest,T_Note_10yr,CPI_item_cost,CPI_purchase_power
2023-04-08,4541.0,-0.053386,33485.29,12087.96,4105.02,2.33,80.70,4.83,3.39,301.808,33.1
2023-04-09,2593.0,0.033976,33485.29,12087.96,4105.02,2.33,80.70,4.83,3.39,301.808,33.1
2023-04-10,1642.0,-0.015850,33586.52,12084.35,4109.11,2.34,79.79,4.83,3.41,301.808,33.1
2023-04-11,3562.0,0.093745,33684.79,12031.88,4108.94,2.36,79.79,4.83,3.43,301.808,33.1
2023-04-12,4119.0,0.080882,33646.50,12031.88,4091.95,2.29,79.79,4.83,3.43,301.808,33.1


In [23]:
# Set date as first column for ease of export to PostgreSQL
df_final = df_final.reset_index()
df_final = df_final.rename(columns={'index':'date'})
df_final

,date,Mentions,Polarity,Dow,NASDAQ,SP500,Inflation_5yr,Oil,Interest,T_Note_10yr,CPI_item_cost,CPI_purchase_power
0,2023-04-08,4541.0,-0.053386,33485.29,12087.96,4105.02,2.33,80.70,4.83,3.39,301.808,33.1
1,2023-04-09,2593.0,0.033976,33485.29,12087.96,4105.02,2.33,80.70,4.83,3.39,301.808,33.1
2,2023-04-10,1642.0,-0.015850,33586.52,12084.35,4109.11,2.34,79.79,4.83,3.41,301.808,33.1
3,2023-04-11,3562.0,0.093745,33684.79,12031.88,4108.94,2.36,79.79,4.83,3.43,301.808,33.1
4,2023-04-12,4119.0,0.080882,33646.50,12031.88,4091.95,2.29,79.79,4.83,3.43,301.808,33.1


#### Note: Intermediate code not included here to update an AWS RDS PostgreSQL database daily with data generated by above code.

### Visualize data with Plotly Dash

In [24]:
import dash
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from jupyter_dash import JupyterDash

In [ ]:
# Visualization with charts for # of relevant tweets per day, polarity, and FRED economic metrics
# Features include FRED economic metrics dropdown bar, time range button selector and date slider at bottom

fig = make_subplots(rows = 3,
    cols = 1,
    subplot_titles = ('# of Tweets', 'Polarity', 'Economic Metrics'),
    shared_xaxes = True)

fig.append_trace(go.Scatter(
    x = df_final['date'],
    y = df_final['Mentions'],
    name = '# of Tweets'), row = 1, col = 1)

fig.add_trace(go.Scatter(
    x = df_final['date'],
    y = df_final['Polarity'],
    name = 'Polarity'), row = 2, col = 1)

fig.append_trace(go.Scatter(
    x = df_final['date'],
    y = df_final['Dow'],
    name = 'Dow'), row = 3, col = 1)

# app.layout = html.Div([html.H2('Twitter Economic Sentiment with FRED Metrics'), 
#                        html.Br(), dcc.Graph(figure = fig)], style = {
#     'width': '75%',
#     'display': 'inline-block',
#     'padding-left': '10%',
#     'padding-right': '15%'
# })

fig.update_layout(height = 700,
                  width = 1100)# title_text = "Twitter Economic Sentiment with FRED Metrics")

# Add date range selector buttons - starter code from Plotly documentation
fig.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = list([
                dict(count = 1,
                    label = "1m",
                    step = "month",
                    stepmode = "backward"),
                dict(count = 6,
                    label = "6m",
                    step = "month",
                    stepmode = "backward"),
                dict(count = 1,
                    label = "YTD",
                    step = "year",
                    stepmode = "todate"),
                dict(count = 1,
                    label = "1y",
                    step = "year",
                    stepmode = "backward"),
                dict(step = "all")
            ]), x = 1.05
        ),
        type = "date"
    )
)

# fig.update_xaxes(matches = 'x')
fig.update_layout(xaxis_showticklabels = True,
    xaxis2_showticklabels = True,
    xaxis3_showticklabels = True)

updatemenu = []
buttons = []

# button with one option for each dataframe column excluding first two df columns
for col in df_final.columns[2: ]:
    buttons.append(dict(method = 'restyle',
        label = col,
        visible = True,
        args = [{
                'y': [df_final[col]],
                'x': [df_final['date']],
                'type': 'scatter'
            },
            [2]
        ],
    ))

# some adjustments to the updatemenus
updatemenu = []
your_menu = dict()
updatemenu.append(your_menu)

updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['showactive'] = True# Position dropdown: https: //stackoverflow.com/questions/50330544/positioning-a-dropdown-in-plotly
updatemenu[0]['x'] = 1.25
updatemenu[0]['y'] = 0.22

# add dropdown menus to the figure
fig.update_layout(showlegend = False, updatemenus = updatemenu)

# Add date range selector slider
fig.update_layout(xaxis3 = dict(rangeslider = dict(visible = True), type = 'date'))

app = Dash(__name__, external_stylesheets = [dbc.themes.BOOTSTRAP])


app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
                html.H1("Sentiment Analysis of the U.S. Economy with Twitter"), 
                html.H3("By: Mushfiqur Rahman")
        ],
            width = 'auto',
            style = {
                'padding': '7px',
                'backgroundColor': '#f0e442',
                'align': 'center',
                'width': '90%',
                "justify": "center",
                'textAlign': 'center'
            })
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Graph(id = 'plot', figure = fig,
                style = {
                    "width": "80.8%",
                    #"height": "800px",
                    "display": "inline-block",
                    #"border": "2px #3c1c1c solid",
                    #"padding-top": "0px",
                    #"padding-left": "10px",
                    #"overflow": "hidden",
                    #"top": "200%",
                    #"left": "20%",
                    "align": "center",
                    "justify": "center",
                    'horrizontal-align': 'center',
                    #"margin-top": "-400px"
                })
        ], width = 'auto')
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Markdown('''
                **Overview:**  
                The goal for this project is to conduct sentiment analyses of the US economy using 
                Twitter data.  Scraped Tweets are fed into the Sentiment Analysis library TextBlob. 
                Currently, elevated Twitter access only allows searching for the past 7 days of Tweets.
                Initial questions to explore: What are the current overall sentiments of the economy? 
                Are they positive or negative or neutral? How has it varied over time? 
                Is there a correlation between economic sentiment and perhaps certain economic metrics 
                such as stock market performance?  Economic data is downloaded from the U.S. Federal 
                Reserve's FRED API.  In the future once enough data is collected, the data will be 
                fed into scikit-learn machine learning models to best fit a model and determine if any 
                of the economic indicators have correlations with sentiment polarity as the target variable.
                                
                **Methodology:**  
                1. Download tweets mentioning the U.S. Economy, determine tweet counts, assess polarity by day  
                2. Download important economic metrics from the FRED API  
                3. Create a pandas dataframe with combined data from both APIs  
                4. Visualize data with Plotly Dash  
                5. Deploy as a website using AWS EC2, RDBMS etc...  
                
                **Skills:**  
                Python, Twitter API, FRED API, Tweepy, Pandas, TextBlob, Matplotlib, Seaborn, 
                Plotly Dash, AWS, Data Munging.
                
                **Resources:**  
                Link to portfolio for the full code in Jupyter Notebook:  
                &emsp;https://github.com/lflrnr/Portfolio/tree/main/3.%20Economy%20and%20Twitter  
                Twitter API docs  
                &emsp;https://developer.twitter.com/en/docs/twitter-api/  
                Federal Reserve Bank of St. Louis - FRED API docs  
                &emsp;https://fred.stlouisfed.org/docs/api/fred/
                
                **Disclaimers:**  
                "This product uses the FRED® API but is not endorsed or certified by 
                the Federal Reserve Bank of St. Louis."  
                Furthermore, this exercise is for demonstration purposes only. 
                This dashboard, the underlying code, technologies involved and 
                their related organizations and the author does not assume any liability 
                for the use or misuse of any of the aforementioned 
                resources nor claim their veracity, timeliness, or conclusions stated or inferred.
                
                ''',
                style = {
                    'width': '85%',
                    "padding-left": "50px",
                    'fontSize':18
                })
        ], width = 'auto')
    ], justify = 'center')
])


# if __name__ == '__main__':
#     app.run_server(debug = False, use_reloader = False) # turn off inline to get url " mode='inline' "

if __name__ == '__main__':
    app.run_server()
    display()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Apr/2023 00:06:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 00:06:39] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Apr/2023 00:06:39] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Apr/2023 00:06:39] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Apr/2023 00:06:39] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Apr/2023 00:06:39] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 00:06:39] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 00:06:39] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Apr/2023 00:06:39] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Apr/2023 00:06:39] "GET /_dash-component-suites/dash/dcc/async-